In [28]:
from memory.unsafe import DTypePointer
from memory import memset_zero

struct Field[DT: DType, VW: Int]:
    var data: DTypePointer[DT]
    var gx: Int
    var gy: Int
    var gz: Int
    var dsize: Int
    
    fn __init__(inout self, gx: Int, gy: Int, gz: Int):
        self.dsize = (gx+2)*(gy+2)*(gz+2)
        self.data = DTypePointer[DT].alloc(self.dsize)
        memset_zero(self.data, self.dsize)
        self.gx = gx
        self.gy = gy
        self.gz = gz

    fn __copyinit__(inout self, other: Self):
        self.gx = other.gx
        self.gy = other.gy
        self.gz = other.gz
        self.data = DTypePointer[DT].alloc(other.dsize)
        self.dsize = other.dsize
        for i in range(0,self.dsize - self.dsize % VW,VW):
            self.data.simd_store[VW](i, other.data.simd_load[VW](i))
        for i in range(self.dsize - self.dsize % VW, self.dsize):
            self.data.simd_store[1](i, other.data.simd_load[1](i))

    fn __add__(self, rhs: Field[DT,VW]) raises -> Field[DT,VW]:
        # test if the fields have the same sizes
        if self.gx != rhs.gx or self.gy != rhs.gy or self.gz != rhs.gz :
            raise ("cannot add fields with different grid sizes")
        let result = Field[DT,VW](self.gx, self.gy, self.gz)
        let p : Int
        let vwgx = self.gx - self.gx%VW
        for k in range(1,self.gz+1):
            for j in range(1,self.gy+1):
                for i in range (1,vwgx+1,VW):
                    p = k * (self.gx+2) * (self.gy+2) + j * (self.gx+2) + i
                    result.data.simd_store[VW](p, self.data.simd_load[VW](p) + rhs.data.simd_load[VW](p))
                for i in range (vwgx+1,self.gx+1):
                    p = k * (self.gx+2) * (self.gy+2) + j * (self.gx+2) + i
                    result.data.simd_store[1](p, self.data.simd_load[1](p) + rhs.data.simd_load[1](p))
        return result

    fn __del__(owned self):
        self.data.free()
        print("done del")
        
    fn zero(inout self):
        memset_zero[DT](self.data, self.dsize)

    @always_inline
    fn __getitem__(self, x: Int, y: Int, z: Int) -> SIMD[DT,1]:
        let p = z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x 
        return self.data.load(p)

    #@always_inline
    #fn load[nelts:Int=VW](self, x: Int, y: Int, z: Int) -> SIMD[DT,nelts]:
    #    return self.data.simd_load[nelts](z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x)

    @always_inline
    fn __setitem__(self, x: Int, y: Int, z: Int, val: SIMD[DT,1]):
        return self.data.store(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x, val)

    #@always_inline
    #fn store[nelts:Int=VW](self, x: Int, y: Int, z: Int, val: SIMD[DT, nelts]):
    #    self.data.simd_store(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x, val)

In [30]:

let n=8
alias w=4
var a = Field[DType.float32,w](n,1,1)
var b = Field[DType.float32,w](n,1,1)
#var c = Field[DType.float32,w](n,1,1)

for i in range(1,n+1):
    a[i,1,1] = i
    b[i,1,1] = i*10
    print(i, a[i,1,1], b[i,1,1])

let c : Field[DType.float32,w] = a + b
#c.zero()
for i in range(1,n+1):
    print(i, c[i,1,1])



1 1.0 10.0
2 2.0 20.0
3 3.0 30.0
4 4.0 40.0
5 5.0 50.0
6 6.0 60.0
7 7.0 70.0
8 8.0 80.0
done del
1 11.0
2 22.0
3 33.0
4 44.0
5 55.0
6 66.0
7 77.0
8 88.0


In [16]:
var a: SIMD[DType.float64, 4]
var b: SIMD[DType.float64, 4]
var c: SIMD[DType.float64, 4]

a=1.0
b=3.33
c=a+b
print(a,b,c)

[1.0, 1.0, 1.0, 1.0] [3.3300000000000001, 3.3300000000000001, 3.3300000000000001, 3.3300000000000001] [4.3300000000000001, 4.3300000000000001, 4.3300000000000001, 4.3300000000000001]
